In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Statistical analysis
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Machine learning (supervised learning)
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

# Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_1 = pd.read_excel("2021.xlsx")
df_2 = pd.read_excel("2022.xlsx")
df_concatenated = pd.concat([df_1, df_2], ignore_index=True)
# Display the first few rows of the concatenated DataFrame
print(df_concatenated.head())
# Display the shape of the concatenated DataFrame
print(f"Shape of concatenated DataFrame: {df_concatenated.shape}")
# Display the columns of the concatenated DataFrame
print(f"Columns in concatenated DataFrame: {df_concatenated.columns.tolist()}")
# Display the data types of the columns in the concatenated DataFrame
print(f"Data types of columns in concatenated DataFrame:\n{df_concatenated.dtypes}")
# Display summary statistics of the concatenated DataFrame
print(f"Summary statistics of concatenated DataFrame:\n{df_concatenated.describe()}")
# Check for missing values in the concatenated DataFrame
print(f"Missing values in concatenated DataFrame:\n{df_concatenated.isnull().sum()}")

In [ ]:
# Step 1: Data Preprocessing and Initial Exploration
# Building on your concatenated dataset

# First, let's standardize column names to English for easier processing
df_concatenated.columns = ['sales_month', 'primary_platform', 'secondary_platform', 
                          'store_name', 'brand_name', 'product_code', 
                          'sales_amount', 'sales_quantity']

platform_mapping = {
    '抖音': 'Douyin',
    '京东': 'JD',
    '天猫': 'Tmall'
}
# Map primary_platform to English names
df_concatenated['primary_platform'] = df_concatenated['primary_platform'].map(platform_mapping)

# Convert sales_month to datetime
df_concatenated['sales_month'] = pd.to_datetime(df_concatenated['sales_month'])

# Add year column for analysis
df_concatenated['year'] = df_concatenated['sales_month'].dt.year

print("=== BASIC DATA PREPROCESSING ===")
print(f"Date range: {df_concatenated['sales_month'].min()} to {df_concatenated['sales_month'].max()}")
print(f"Years covered: {sorted(df_concatenated['year'].unique())}")
print(f"Platforms: {df_concatenated['primary_platform'].unique()}")

# Platform distribution analysis
print("\n=== PLATFORM COVERAGE ANALYSIS ===")
platform_coverage = df_concatenated.groupby(['primary_platform', 'year']).agg({
    'sales_month': ['min', 'max', 'nunique'],
    'sales_quantity': ['count', 'sum'],
    'store_name': 'nunique',
    'brand_name': 'nunique'
}).round(2)

platform_coverage.columns = ['_'.join(col) for col in platform_coverage.columns]
print(platform_coverage)

# Missing data analysis
print("\n=== MISSING DATA ANALYSIS ===")
missing_by_platform = df_concatenated.groupby('primary_platform').apply(
    lambda x: x.isnull().sum()
)
print("Missing data by platform:")
print(missing_by_platform)

# Handle missing values strategically
print("\n=== MISSING DATA HANDLING ===")
# For missing store names, we'll create a placeholder
df_concatenated['store_name'] = df_concatenated['store_name'].fillna('Unknown_Store')

# For missing brand names, let's analyze the pattern first
missing_brands = df_concatenated[df_concatenated['brand_name'].isnull()]
print(f"Records with missing brand names: {len(missing_brands)}")
print("Platform distribution of missing brands:")
print(missing_brands['primary_platform'].value_counts())

# Fill missing brand names with 'Unknown_Brand'
df_concatenated['brand_name'] = df_concatenated['brand_name'].fillna('Unknown_Brand')

print(f"After handling missing data: {df_concatenated.isnull().sum().sum()} missing values remain")

# Data quality checks
print("\n=== DATA QUALITY ANALYSIS ===")

# Check for zero/negative sales
zero_sales = df_concatenated[df_concatenated['sales_quantity'] <= 0]
print(f"Records with zero or negative sales quantity: {len(zero_sales)} ({len(zero_sales)/len(df_concatenated)*100:.2f}%)")

zero_amount = df_concatenated[df_concatenated['sales_amount'] <= 0]
print(f"Records with zero or negative sales amount: {len(zero_amount)} ({len(zero_amount)/len(df_concatenated)*100:.2f}%)")

# Price analysis (sales_amount / sales_quantity)
df_concatenated['unit_price'] = df_concatenated['sales_amount'] / df_concatenated['sales_quantity']
df_concatenated['unit_price'] = df_concatenated['unit_price'].replace([np.inf, -np.inf], np.nan)

print(f"\nUnit price statistics:")
print(df_concatenated['unit_price'].describe())

# Extreme value analysis
print("\n=== EXTREME VALUES ANALYSIS ===")
q99 = df_concatenated['sales_quantity'].quantile(0.99)
q95 = df_concatenated['sales_quantity'].quantile(0.95)
q90 = df_concatenated['sales_quantity'].quantile(0.90)

print(f"Sales quantity percentiles:")
print(f"90th percentile: {q90:,.0f}")
print(f"95th percentile: {q95:,.0f}")
print(f"99th percentile: {q99:,.0f}")
print(f"Maximum: {df_concatenated['sales_quantity'].max():,.0f}")

# Identify extreme outliers (>99th percentile)
extreme_outliers = df_concatenated[df_concatenated['sales_quantity'] > q99]
print(f"\nExtreme outliers (>99th percentile): {len(extreme_outliers)} records")
print("Top 5 extreme sales:")
print(extreme_outliers.nlargest(5, 'sales_quantity')[['sales_month', 'primary_platform', 'store_name', 'brand_name', 'sales_quantity']])

# Monthly sales distribution
print("\n=== TEMPORAL DISTRIBUTION ANALYSIS ===")
monthly_totals = df_concatenated.groupby(['year', df_concatenated['sales_month'].dt.month]).agg({
    'sales_quantity': 'sum',
    'sales_amount': 'sum'
}).round(0)

monthly_totals.columns = ['total_quantity', 'total_amount']
print("Monthly sales totals by year:")
print(monthly_totals.head(10))

# Platform comparison
print("\n=== PLATFORM PERFORMANCE COMPARISON ===")
platform_stats = df_concatenated.groupby('primary_platform').agg({
    'sales_quantity': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    'sales_amount': ['sum', 'mean'],
    'store_name': 'nunique',
    'brand_name': 'nunique',
    'unit_price': 'mean'
}).round(2)

platform_stats.columns = ['_'.join(col) for col in platform_stats.columns]
print(platform_stats)

# Save processed data for next steps
print("\n=== DATA READY FOR FEATURE ENGINEERING ===")
print(f"Final dataset shape: {df_concatenated.shape}")
print(f"Date range: {df_concatenated['sales_month'].min()} to {df_concatenated['sales_month'].max()}")
print(f"Total platforms: {df_concatenated['primary_platform'].nunique()}")
print(f"Total stores: {df_concatenated['store_name'].nunique()}")
print(f"Total brands: {df_concatenated['brand_name'].nunique()}")
print(f"Total products: {df_concatenated['product_code'].nunique()}")

In [ ]:
# Step 2: Advanced Temporal Feature Engineering
# Building sophisticated time-based features to capture customer behavior and seasonal patterns

def create_advanced_temporal_features(df):
    """
    Create advanced temporal features based on the patterns observed in preprocessing
    """
    df = df.copy()
    
    print("=== CREATING ADVANCED TEMPORAL FEATURES ===")
    
    # Sort data properly for time series operations
    df = df.sort_values(['primary_platform', 'store_name', 'brand_name', 'sales_month'])
    
    # Basic temporal components
    df['month'] = df['sales_month'].dt.month
    df['quarter'] = df['sales_month'].dt.quarter
    df['year'] = df['sales_month'].dt.year
    df['month_year'] = df['sales_month'].dt.to_period('M')
    
    # Days since start of dataset (for trend analysis)
    start_date = df['sales_month'].min()
    df['days_since_start'] = (df['sales_month'] - start_date).dt.days
    
    print("✓ Basic temporal components created")
    
    # Cyclical encoding for temporal features (crucial for neural networks)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)
    
    print("✓ Cyclical encoding applied")
    
    # Chinese e-commerce calendar features (based on your domain knowledge)
    promotional_months = [1, 6, 9, 11, 12]  # From your previous analysis
    
    df['is_promotional'] = df['month'].isin(promotional_months).astype(int)
    
    # Distance to promotional periods (helps model anticipate events)
    def calculate_distance_to_events(month, event_months):
        if not event_months:
            return 6  # Maximum distance
        distances = []
        for event_month in event_months:
            # Calculate circular distance (considering year wraparound)
            distance = min(abs(month - event_month), 12 - abs(month - event_month))
            distances.append(distance)
        return min(distances)
    
    df['distance_to_promo'] = df['month'].apply(lambda x: calculate_distance_to_events(x, promotional_months))
    
    print("✓ Chinese e-commerce calendar features created")
    
    # Data-driven seasonal intensity (learning from actual data patterns)
    monthly_baseline = df.groupby('month')['sales_quantity'].mean()
    overall_mean = df['sales_quantity'].mean()
    monthly_intensity = (monthly_baseline / overall_mean).to_dict()
    
    df['monthly_intensity_learned'] = df['month'].map(monthly_intensity)
    
    # Platform-specific seasonal patterns
    platform_monthly_patterns = df.groupby(['primary_platform', 'month'])['sales_quantity'].mean()
    platform_overall_means = df.groupby('primary_platform')['sales_quantity'].mean()
    
    def get_platform_seasonal_index(row):
        platform = row['primary_platform']
        month = row['month']
        pattern_value = platform_monthly_patterns.get((platform, month), platform_overall_means.get(platform, overall_mean))
        baseline = platform_overall_means.get(platform, overall_mean)
        return pattern_value / baseline if baseline > 0 else 1
    
    df['platform_seasonal_index'] = df.apply(get_platform_seasonal_index, axis=1)
    
    print("✓ Data-driven seasonal patterns learned")
    
    # Year-over-year growth features
    df['is_2022'] = (df['year'] == 2022).astype(int)
    
    # Create year-over-year comparison features
    yoy_comparison = df.groupby(['primary_platform', 'store_name', 'brand_name', 'month']).agg({
        'sales_quantity': 'mean'
    }).reset_index()
    
    yoy_2021 = yoy_comparison[yoy_comparison['month'].isin([4,5,6,7,8,9,10,11,12])].copy()  # Douyin available months
    yoy_2021['comparison_key'] = yoy_2021['primary_platform'] + '_' + yoy_2021['store_name'] + '_' + yoy_2021['brand_name'] + '_' + yoy_2021['month'].astype(str)
    yoy_baseline = yoy_2021.set_index('comparison_key')['sales_quantity'].to_dict()
    
    df['comparison_key'] = df['primary_platform'] + '_' + df['store_name'] + '_' + df['brand_name'] + '_' + df['month'].astype(str)
    df['yoy_baseline'] = df['comparison_key'].map(yoy_baseline)
    df['yoy_growth_potential'] = np.where(
        (df['year'] == 2022) & (df['yoy_baseline'].notna()),
        df['sales_quantity'] / df['yoy_baseline'],
        1
    )
    
    # Clean up temporary columns
    df = df.drop(['comparison_key', 'yoy_baseline'], axis=1)
    
    print("✓ Year-over-year growth features created")
    
    return df

def create_customer_behavior_features(df):
    """
    Create features that capture customer purchasing behavior patterns
    """
    df = df.copy()
    
    print("\n=== CREATING CUSTOMER BEHAVIOR FEATURES ===")
    
    # Store-level behavior analysis
    store_behavior = df.groupby(['store_name', 'primary_platform']).agg({
        'sales_quantity': ['mean', 'std', 'min', 'max', 'count'],
        'sales_amount': ['mean', 'sum'],
        'unit_price': ['mean', 'std'],
        'brand_name': 'nunique',
        'product_code': 'nunique'
    })
    
    # Flatten column names
    store_behavior.columns = ['_'.join(col) for col in store_behavior.columns]
    store_behavior = store_behavior.reset_index()
    
    # Create derived behavior metrics
    store_behavior['store_sales_cv'] = (
        store_behavior['sales_quantity_std'] / store_behavior['sales_quantity_mean']
    ).fillna(0)  # Coefficient of variation - measures consistency
    
    store_behavior['store_sales_range'] = (
        store_behavior['sales_quantity_max'] - store_behavior['sales_quantity_min']
    )  # Sales volatility
    
    store_behavior['avg_revenue_per_transaction'] = (
        store_behavior['sales_amount_mean'] / store_behavior['sales_quantity_mean']
    ).fillna(0)
    
    store_behavior['brand_diversity'] = store_behavior['brand_name_nunique']
    store_behavior['product_diversity'] = store_behavior['product_code_nunique']
    
    # Price positioning
    store_behavior['price_premium_index'] = (
        store_behavior['unit_price_mean'] / store_behavior['unit_price_mean'].mean()
    ).fillna(1)
    
    # Store size categorization based on sales volume
    store_behavior['total_historical_sales'] = store_behavior['sales_quantity_count'] * store_behavior['sales_quantity_mean']
    store_behavior['store_size_category'] = pd.qcut(
        store_behavior['total_historical_sales'], 
        q=5, 
        labels=['Micro', 'Small', 'Medium', 'Large', 'Mega'],
        duplicates='drop'
    )
    
    # Merge back to main dataset
    behavior_cols = ['store_name', 'primary_platform', 'store_sales_cv', 'store_sales_range',
                    'avg_revenue_per_transaction', 'brand_diversity', 'product_diversity',
                    'price_premium_index', 'store_size_category']
    
    df = df.merge(store_behavior[behavior_cols], on=['store_name', 'primary_platform'], how='left')
    
    print("✓ Store behavior features created")
    
    # Brand market positioning features
    brand_positioning = df.groupby(['brand_name', 'primary_platform']).agg({
        'sales_quantity': ['mean', 'sum', 'count'],
        'unit_price': 'mean',
        'store_name': 'nunique'
    })
    
    brand_positioning.columns = ['_'.join(col) for col in brand_positioning.columns]
    brand_positioning = brand_positioning.reset_index()
    
    # Brand market share on each platform
    platform_totals = df.groupby('primary_platform')['sales_quantity'].sum()
    
    def calculate_brand_market_share(row):
        platform = row['primary_platform']
        brand_total = brand_positioning[
            (brand_positioning['brand_name'] == row['brand_name']) & 
            (brand_positioning['primary_platform'] == platform)
        ]['sales_quantity_sum'].iloc[0] if len(brand_positioning[
            (brand_positioning['brand_name'] == row['brand_name']) & 
            (brand_positioning['primary_platform'] == platform)
        ]) > 0 else 0
        
        platform_total = platform_totals.get(platform, 1)
        return brand_total / platform_total if platform_total > 0 else 0
    
    # Simplified market share calculation
    brand_market_share = df.groupby(['brand_name', 'primary_platform'])['sales_quantity'].sum().reset_index()
    platform_totals_dict = df.groupby('primary_platform')['sales_quantity'].sum().to_dict()
    
    brand_market_share['brand_market_share'] = brand_market_share.apply(
        lambda x: x['sales_quantity'] / platform_totals_dict.get(x['primary_platform'], 1), axis=1
    )
    
    df = df.merge(
        brand_market_share[['brand_name', 'primary_platform', 'brand_market_share']], 
        on=['brand_name', 'primary_platform'], 
        how='left'
    )
    
    print("✓ Brand positioning features created")
    
    return df

# Apply the feature engineering
print("Starting Step 2: Advanced Temporal Feature Engineering")
print("=" * 60)

# Create the features (run this in your notebook)  
df_with_temporal = create_advanced_temporal_features(df_concatenated)
df_with_behavior = create_customer_behavior_features(df_with_temporal)

print(f"\nFeature engineering complete!")
print(f"Original columns: {len(df_concatenated.columns)}")
print(f"After temporal features: {len(df_with_temporal.columns)}")
print(f"After behavior features: {len(df_with_behavior.columns)}")
print(f"New features added: {len(df_with_behavior.columns) - len(df_concatenated.columns)}")

# Display new feature categories
temporal_features = [col for col in df_with_behavior.columns if any(x in col.lower() for x in 
    ['month', 'quarter', 'sin', 'cos', 'promotional', 'festival', 'distance', 'intensity', 'seasonal', 'yoy'])]

behavior_features = [col for col in df_with_behavior.columns if any(x in col.lower() for x in 
    ['store_', 'brand_', 'diversity', 'premium', 'market_share', 'size_category'])]

print(f"\nTemporal Features ({len(temporal_features)}):")
for feature in temporal_features:
    print(f"  • {feature}")

print(f"\nBehavior Features ({len(behavior_features)}):")  
for feature in behavior_features:
    print(f"  • {feature}")

print("\n" + "="*60)
print("Ready for Step 3: Lag Features and Temporal Dependencies")

In [ ]:
# Step 3: Advanced Lag Features and Temporal Dependencies
# Creating sophisticated time-series features to capture momentum, trends, and temporal patterns

def create_lag_and_rolling_features(df):
    """
    Create advanced lag and rolling window features for temporal dependencies
    Focuses on store-brand-platform combinations to capture specific patterns
    """
    df = df.copy()
    print("=== CREATING LAG AND ROLLING FEATURES ===")
    
    # Ensure proper sorting for time series operations
    df = df.sort_values(['primary_platform', 'store_name', 'brand_name', 'sales_month'])
    
    # Create unique identifier for store-brand-platform combinations
    df['store_brand_platform_key'] = (
        df['store_name'].astype(str) + '_' + 
        df['brand_name'].astype(str) + '_' + 
        df['primary_platform'].astype(str)
    )
    
    print("✓ Created store-brand-platform grouping keys")
    
    # Lag features - Different time horizons for different business insights
    lag_periods = [1, 2, 3, 6, 12]  # 1-12 months back
    
    for lag in lag_periods:
        df[f'sales_lag_{lag}'] = df.groupby('store_brand_platform_key')['sales_quantity'].shift(lag)
        print(f"✓ Created {lag}-month lag features")
    
    # Rolling window features - Capture trends and volatility
    rolling_windows = [3, 6, 12]  # 3, 6, 12 month windows
    
    for window in rolling_windows:
        # Rolling mean (trend)
        df[f'sales_rolling_mean_{window}'] = (
            df.groupby('store_brand_platform_key')['sales_quantity']
            .rolling(window=window, min_periods=1)
            .mean()
            .reset_index(level=0, drop=True)
        )
        
        # Rolling standard deviation (volatility)
        df[f'sales_rolling_std_{window}'] = (
            df.groupby('store_brand_platform_key')['sales_quantity']
            .rolling(window=window, min_periods=1)
            .std()
            .reset_index(level=0, drop=True)
        ).fillna(0)
        
        # Rolling min and max (range patterns)
        df[f'sales_rolling_min_{window}'] = (
            df.groupby('store_brand_platform_key')['sales_quantity']
            .rolling(window=window, min_periods=1)
            .min()
            .reset_index(level=0, drop=True)
        )
        
        df[f'sales_rolling_max_{window}'] = (
            df.groupby('store_brand_platform_key')['sales_quantity']
            .rolling(window=window, min_periods=1)
            .max()
            .reset_index(level=0, drop=True)
        )
        
        print(f"✓ Created {window}-month rolling window features")
    
    # Momentum and trend features
    print("Creating momentum and trend features...")
    
    # Percentage change features (growth rates)
    for period in [1, 3, 6]:
        df[f'sales_pct_change_{period}'] = (
            df.groupby('store_brand_platform_key')['sales_quantity']
            .pct_change(periods=period)
            .fillna(0)
        )
    
    # Momentum indicators (comparing different time horizons)
    df['sales_momentum_short'] = np.where(
        df['sales_rolling_mean_3'] != 0,
        df['sales_rolling_mean_6'] / df['sales_rolling_mean_3'],
        1
    )  # 6-month trend vs 3-month trend
    
    df['sales_momentum_long'] = np.where(
        df['sales_rolling_mean_6'] != 0,
        df['sales_rolling_mean_12'] / df['sales_rolling_mean_6'],
        1
    )  # 12-month trend vs 6-month trend
    
    # Acceleration (second derivative - change in growth rate)
    df['sales_acceleration'] = (
        df['sales_pct_change_1'] - 
        df.groupby('store_brand_platform_key')['sales_pct_change_1'].shift(1)
    ).fillna(0)
    
    # Volatility indicators
    df['sales_volatility_ratio'] = np.where(
        df['sales_rolling_mean_6'] != 0,
        df['sales_rolling_std_6'] / df['sales_rolling_mean_6'],
        0
    )  # Coefficient of variation over 6 months
    
    print("✓ Created momentum, trend, and volatility features")
    
    return df

def create_cross_platform_dynamics(df):
    """
    Create features that capture competitive dynamics and cross-platform effects
    """
    df = df.copy()
    print("\n=== CREATING CROSS-PLATFORM DYNAMICS ===")
    
    # Monthly platform competition intensity
    monthly_platform_stats = df.groupby(['sales_month', 'primary_platform']).agg({
        'brand_name': 'nunique',
        'store_name': 'nunique', 
        'sales_quantity': ['sum', 'mean', 'count']
    })
    
    monthly_platform_stats.columns = ['_'.join(col) for col in monthly_platform_stats.columns]
    monthly_platform_stats = monthly_platform_stats.reset_index()
    
    # Merge competition metrics
    df = df.merge(
        monthly_platform_stats.rename(columns={
            'brand_name_nunique': 'monthly_competing_brands',
            'store_name_nunique': 'monthly_competing_stores',
            'sales_quantity_sum': 'monthly_platform_total_sales',
            'sales_quantity_mean': 'monthly_platform_avg_sales',
            'sales_quantity_count': 'monthly_platform_transactions'
        }),
        on=['sales_month', 'primary_platform'],
        how='left'
    )
    
    # Brand performance across platforms (for brands present on multiple platforms)
    brand_platform_presence = df.groupby('brand_name')['primary_platform'].nunique()
    multi_platform_brands = brand_platform_presence[brand_platform_presence > 1].index
    
    df['is_multi_platform_brand'] = df['brand_name'].isin(multi_platform_brands).astype(int)
    
    # For multi-platform brands, calculate relative performance
    brand_platform_performance = df.groupby(['brand_name', 'primary_platform'])['sales_quantity'].mean()
    brand_overall_performance = df.groupby('brand_name')['sales_quantity'].mean()
    
    def calculate_platform_preference_score(row):
        if row['is_multi_platform_brand'] == 0:
            return 1.0  # Single platform brands get neutral score
        
        brand = row['brand_name']
        platform = row['primary_platform']
        
        platform_performance = brand_platform_performance.get((brand, platform), 0)
        overall_performance = brand_overall_performance.get(brand, 1)
        
        return platform_performance / overall_performance if overall_performance > 0 else 1.0
    
    df['brand_platform_preference_score'] = df.apply(calculate_platform_preference_score, axis=1)
    
    print("✓ Created cross-platform competitive dynamics")
    
    return df

def create_seasonal_interaction_features(df):
    """
    Create advanced seasonal and promotional interaction features
    """
    df = df.copy()
    print("\n=== CREATING SEASONAL INTERACTION FEATURES ===")
    
    # Brand-seasonal performance patterns
    brand_seasonal_performance = df.groupby(['brand_name', 'month'])['sales_quantity'].mean()
    brand_annual_performance = df.groupby('brand_name')['sales_quantity'].mean()
    
    def get_brand_seasonal_index(row):
        brand = row['brand_name']
        month = row['month']
        
        seasonal_perf = brand_seasonal_performance.get((brand, month), 0)
        annual_perf = brand_annual_performance.get(brand, 1)
        
        return seasonal_perf / annual_perf if annual_perf > 0 else 1.0
    
    df['brand_seasonal_index'] = df.apply(get_brand_seasonal_index, axis=1)
    
    # Platform-seasonal interaction
    platform_seasonal_performance = df.groupby(['primary_platform', 'month'])['sales_quantity'].mean()
    platform_annual_performance = df.groupby('primary_platform')['sales_quantity'].mean()
    
    def get_platform_seasonal_index(row):
        platform = row['primary_platform']
        month = row['month']
        
        seasonal_perf = platform_seasonal_performance.get((platform, month), 0)
        annual_perf = platform_annual_performance.get(platform, 1)
        
        return seasonal_perf / annual_perf if annual_perf > 0 else 1.0
    
    df['platform_seasonal_index'] = df.apply(get_platform_seasonal_index, axis=1)
    
    # Promotional effectiveness by brand
    promo_performance = df[df['is_promotional'] == 1].groupby('brand_name')['sales_quantity'].mean()
    non_promo_performance = df[df['is_promotional'] == 0].groupby('brand_name')['sales_quantity'].mean()
    
    promotional_lift = (promo_performance / non_promo_performance).fillna(1.0)
    df['brand_promotional_effectiveness'] = df['brand_name'].map(promotional_lift).fillna(1.0)
    
    # Complex interactions
    df['brand_seasonal_promo_interaction'] = (
        df['brand_seasonal_index'] * 
        df['is_promotional'] * 
        df['brand_promotional_effectiveness']
    )
    
    df['platform_brand_seasonal_interaction'] = (
        df['platform_seasonal_index'] * 
        df['brand_seasonal_index']
    )
    
    # Trend-season interaction (are trends stronger in certain seasons?)
    df['trend_seasonal_interaction'] = (
        df['sales_pct_change_3'] * df['monthly_intensity_learned']
    )
    
    print("✓ Created seasonal interaction features")
    
    return df

def create_outlier_and_spike_features(df):
    """
    Create features to help models handle extreme outliers and sales spikes
    """
    df = df.copy()
    print("\n=== CREATING OUTLIER AND SPIKE DETECTION FEATURES ===")
    
    # Historical spike detection for each store-brand combination
    spike_threshold_99 = df['sales_quantity'].quantile(0.99)
    spike_threshold_95 = df['sales_quantity'].quantile(0.95)
    
    df['is_extreme_spike'] = (df['sales_quantity'] > spike_threshold_99).astype(int)
    df['is_major_spike'] = (df['sales_quantity'] > spike_threshold_95).astype(int)
    
    # Store-brand spike history
    spike_history = df.groupby('store_brand_platform_key').agg({
        'is_extreme_spike': 'sum',
        'is_major_spike': 'sum'
    }).rename(columns={
        'is_extreme_spike': 'historical_extreme_spikes',
        'is_major_spike': 'historical_major_spikes'
    })
    
    df = df.merge(spike_history, on='store_brand_platform_key', how='left')
    
    # Spike propensity score
    total_records_per_key = df.groupby('store_brand_platform_key').size()
    df['spike_propensity'] = df['historical_major_spikes'] / df.groupby('store_brand_platform_key').cumcount().add(1)
    
    # Distance from normal behavior
    df['deviation_from_rolling_mean'] = np.where(
        df['sales_rolling_mean_6'] > 0,
        (df['sales_quantity'] - df['sales_rolling_mean_6']) / df['sales_rolling_mean_6'],
        0
    )
    
    # Z-score based on rolling statistics  
    df['rolling_z_score'] = np.where(
        df['sales_rolling_std_6'] > 0,
        (df['sales_quantity'] - df['sales_rolling_mean_6']) / df['sales_rolling_std_6'],
        0
    )
    
    print("✓ Created outlier and spike detection features")
    
    return df

# Master function to create all advanced features
def create_all_advanced_features(df):
    """
    Apply all advanced feature engineering steps
    """
    print("=" * 60)
    print("STEP 3: ADVANCED TEMPORAL DEPENDENCIES & INTERACTIONS")
    print("=" * 60)
    
    # Apply all feature engineering steps
    df = create_lag_and_rolling_features(df)
    df = create_cross_platform_dynamics(df)
    df = create_seasonal_interaction_features(df)
    df = create_outlier_and_spike_features(df)
    
    # Clean up intermediate columns
    cleanup_cols = ['store_brand_platform_key']
    df = df.drop(columns=[col for col in cleanup_cols if col in df.columns])
    
    print("\n" + "=" * 60)
    print("FEATURE ENGINEERING SUMMARY")
    print("=" * 60)
    
    # Count features by type
    lag_features = [col for col in df.columns if 'lag_' in col]
    rolling_features = [col for col in df.columns if 'rolling_' in col]
    momentum_features = [col for col in df.columns if any(x in col for x in ['momentum', 'acceleration', 'pct_change', 'volatility'])]
    platform_features = [col for col in df.columns if any(x in col for x in ['platform', 'competing', 'multi_platform'])]
    seasonal_features = [col for col in df.columns if any(x in col for x in ['seasonal', 'promotional', 'interaction'])]
    spike_features = [col for col in df.columns if any(x in col for x in ['spike', 'deviation', 'z_score', 'propensity'])]
    
    print(f"Lag Features ({len(lag_features)}): {lag_features}")
    print(f"Rolling Window Features ({len(rolling_features)}): {rolling_features[:5]}..." if len(rolling_features) > 5 else f"Rolling Window Features ({len(rolling_features)}): {rolling_features}")
    print(f"Momentum Features ({len(momentum_features)}): {momentum_features}")
    print(f"Platform Dynamics ({len(platform_features)}): {platform_features}")
    print(f"Seasonal Interactions ({len(seasonal_features)}): {seasonal_features}")
    print(f"Spike Detection ({len(spike_features)}): {spike_features}")
    
    total_engineered_features = len(lag_features) + len(rolling_features) + len(momentum_features) + len(platform_features) + len(seasonal_features) + len(spike_features)
    print(f"\nTotal Advanced Features Created: {total_engineered_features}")
    print(f"Final Dataset Shape: {df.shape}")
    
    return df

# Usage in your notebook:
print("Ready to create advanced temporal dependencies...")
print("Run: df_advanced = create_all_advanced_features(df_with_behavior)")

In [ ]:
# Step 4: Store Categorization and Model Preparation
# Adding store type categorization and preparing for advanced modeling

def add_store_categorization(df):
    """
    Add store type categorization based on Chinese e-commerce naming patterns
    """
    df = df.copy()
    
    print("=== ADDING STORE CATEGORIZATION ===")
    
    def categorize_store(store_name):
        """
        Categorize stores based on Chinese e-commerce store naming conventions
        """
        if pd.isna(store_name):
            return 'Unknown'
        
        store_name = str(store_name)
        
        if '官方旗舰店' in store_name:
            return 'Official Flagship'
        elif '卖场旗舰店' in store_name:
            return 'Mall Flagship'
        elif '旗舰店' in store_name:
            return 'Flagship'
        elif '专卖店' in store_name or '专营店' in store_name:
            return 'Specialty Store'
        elif '卖场店' in store_name:
            return 'Mall Store'
        elif '自营' in store_name:
            return 'Platform Direct'
        elif '超市' in store_name:
            return 'Supermarket'
        else:
            return 'Other'
    
    # Apply categorization
    df['store_type'] = df['store_name'].apply(categorize_store)
    
    # Analyze store type distribution
    store_type_analysis = df.groupby(['store_type', 'primary_platform']).agg({
        'sales_quantity': ['count', 'sum', 'mean'],
        'sales_amount': ['sum', 'mean'],
        'unit_price': 'mean',
        'store_name': 'nunique'
    }).round(2)
    
    store_type_analysis.columns = ['_'.join(col) for col in store_type_analysis.columns]
    print("Store Type Analysis by Platform:")
    print(store_type_analysis)
    
    # Create store type performance features
    store_type_performance = df.groupby('store_type').agg({
        'sales_quantity': 'mean',
        'unit_price': 'mean'
    })
    
    # Store type premium index (compared to average)
    avg_sales = df['sales_quantity'].mean()
    avg_price = df['unit_price'].mean()
    
    store_type_performance['sales_performance_index'] = store_type_performance['sales_quantity'] / avg_sales
    store_type_performance['price_premium_index'] = store_type_performance['unit_price'] / avg_price
    
    # Map back to main dataset
    df['store_type_sales_index'] = df['store_type'].map(store_type_performance['sales_performance_index']).fillna(1)
    df['store_type_price_index'] = df['store_type'].map(store_type_performance['price_premium_index']).fillna(1)
    
    # One-hot encode store types for modeling
    store_type_dummies = pd.get_dummies(df['store_type'], prefix='store_type')
    df = pd.concat([df, store_type_dummies], axis=1)
    
    print(f"✓ Store categorization complete. Created {len(store_type_dummies.columns)} store type features")
    
    return df

def prepare_modeling_dataset(df):
    """
    Prepare the dataset for advanced modeling
    """
    df = df.copy()
    
    print("\n=== PREPARING DATASET FOR MODELING ===")
    
    # Handle any remaining missing values in lag features (expected for early records)
    lag_columns = [col for col in df.columns if 'lag_' in col or 'rolling_' in col]
    
    for col in lag_columns:
        if df[col].isnull().any():
            # Fill NaN lag values with appropriate defaults
            if 'lag_' in col:
                # For lag features, use forward fill within groups, then 0
                df[col] = df.groupby(['primary_platform', 'store_name', 'brand_name'])[col].ffill().fillna(0)
            elif 'rolling_mean' in col:
                # For rolling means, use the current value
                df[col] = df[col].fillna(df['sales_quantity'])
            elif 'rolling_std' in col:
                # For rolling std, use 0 (no volatility)
                df[col] = df[col].fillna(0)
            elif 'rolling_min' in col or 'rolling_max' in col:
                # For rolling min/max, use current value
                df[col] = df[col].fillna(df['sales_quantity'])
    
    # Handle any infinite values
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].replace([np.inf, -np.inf], 0)
    
    # Create final feature list for modeling
    feature_categories = {
        'temporal_basic': ['month', 'quarter', 'year', 'days_since_start'],
        'temporal_cyclical': [col for col in df.columns if any(x in col for x in ['sin', 'cos'])],
        'promotional': [col for col in df.columns if any(x in col for x in ['promotional', 'distance_to_promo'])],
        'seasonal': [col for col in df.columns if 'seasonal' in col and 'interaction' not in col],
        'lag_features': [col for col in df.columns if 'lag_' in col],
        'rolling_features': [col for col in df.columns if 'rolling_' in col],
        'momentum': [col for col in df.columns if any(x in col for x in ['momentum', 'acceleration', 'pct_change', 'volatility'])],
        'store_behavior': [col for col in df.columns if any(x in col for x in ['store_sales_cv', 'store_sales_range', 'brand_diversity', 'product_diversity'])],
        'store_type': [col for col in df.columns if 'store_type_' in col],
        'brand_market': [col for col in df.columns if any(x in col for x in ['brand_market_share', 'brand_promotional_effectiveness'])],
        'platform_dynamics': [col for col in df.columns if any(x in col for x in ['competing', 'multi_platform', 'platform_preference'])],
        'interactions': [col for col in df.columns if 'interaction' in col],
        'spike_detection': [col for col in df.columns if any(x in col for x in ['spike', 'deviation', 'z_score', 'propensity'])]
    }
    
    # Print feature summary
    total_features = 0
    print("\nFEATURE CATEGORIES FOR MODELING:")
    for category, features in feature_categories.items():
        if features:
            print(f"{category.upper()}: {len(features)} features")
            total_features += len(features)
    
    print(f"\nTOTAL MODELING FEATURES: {total_features}")
    
    # Identify target and exclude non-feature columns
    exclude_columns = [
        'sales_month', 'store_name', 'brand_name', 'product_code', 
        'sales_amount', 'sales_quantity', 'unit_price', 'store_type',
        'month_year', 'primary_platform', 'secondary_platform'  # Exclude string columns
    ]
    
    modeling_features = [col for col in df.columns if col not in exclude_columns]
    
    # Ensure we only include numeric features for correlation analysis
    numeric_features = df[modeling_features].select_dtypes(include=[np.number]).columns.tolist()
    
    print(f"FEATURES READY FOR MODELING: {len(modeling_features)}")
    print(f"NUMERIC FEATURES FOR CORRELATION: {len(numeric_features)}")
    print(f"TARGET VARIABLE: sales_quantity")
    
    # Data quality check
    print(f"\nDATA QUALITY CHECK:")
    print(f"Dataset shape: {df.shape}")
    print(f"Missing values in features: {df[modeling_features].isnull().sum().sum()}")
    print(f"Infinite values in numeric features: {np.isinf(df[numeric_features]).sum().sum()}")
    
    # Feature correlation analysis (top correlated features with target) - only numeric features
    target_correlations = df[numeric_features + ['sales_quantity']].corr()['sales_quantity'].abs().sort_values(ascending=False)
    
    print(f"\nTOP 10 FEATURES CORRELATED WITH TARGET:")
    for i, (feature, corr) in enumerate(target_correlations.head(11).items()):
        if feature != 'sales_quantity':  # Skip target itself
            print(f"{i+1:2d}. {feature:<40} {corr:.4f}")
    
    return df, modeling_features

def create_rolling_time_series_splits(df):
    """
    Create rolling time series splits for comprehensive seasonal validation
    Option 2: Multiple train/val splits across different seasons
    
    This approach tests model performance across ALL seasons, not just one quarter,
    providing much more robust validation of seasonal pattern learning.
    """
    print(f"\n=== CREATING ROLLING TIME SERIES SPLITS ===")
    
    # Sort by date to ensure proper time series order
    df = df.sort_values('sales_month')
    
    print(f"Total records: {len(df):,}")
    print(f"Date range: {df['sales_month'].min()} to {df['sales_month'].max()}")
    
    # Create 4 rolling splits to test across all seasons
    splits = []
    
    # Split 1: Train on 2021 full year, Validate on 2022 Q1 (Jan-Mar)
    train_1 = df[df['sales_month'].dt.year == 2021].copy()
    val_1 = df[(df['sales_month'].dt.year == 2022) & 
               (df['sales_month'].dt.month.isin([1,2,3]))].copy()
    splits.append((train_1, val_1, "2021_full → 2022_Q1"))
    
    # Split 2: Train on 2021 + 2022 Q1, Validate on 2022 Q2 (Apr-Jun)
    train_2 = df[df['sales_month'] <= '2022-03-01'].copy()
    val_2 = df[(df['sales_month'].dt.year == 2022) & 
               (df['sales_month'].dt.month.isin([4,5,6]))].copy()
    splits.append((train_2, val_2, "2021+2022Q1 → 2022_Q2"))
    
    # Split 3: Train on 2021 + 2022 H1, Validate on 2022 Q3 (Jul-Sep)  
    train_3 = df[df['sales_month'] <= '2022-06-01'].copy()
    val_3 = df[(df['sales_month'].dt.year == 2022) & 
               (df['sales_month'].dt.month.isin([7,8,9]))].copy()
    splits.append((train_3, val_3, "2021+2022H1 → 2022_Q3"))
    
    # Split 4: Train on 2021 + 2022 Q1-Q3, Validate on 2022 Q4 (Oct-Dec)
    train_4 = df[df['sales_month'] <= '2022-09-01'].copy()
    val_4 = df[(df['sales_month'].dt.year == 2022) & 
               (df['sales_month'].dt.month.isin([10,11,12]))].copy()
    splits.append((train_4, val_4, "2021+2022Q1Q2Q3 → 2022_Q4"))
    
    # Print split information
    print(f"\nROLLING TIME SERIES SPLITS CREATED:")
    print("="*60)
    
    for i, (train, val, description) in enumerate(splits):
        print(f"SPLIT {i+1}: {description}")
        print(f"  Train: {len(train):,} records ({train['sales_month'].min()} to {train['sales_month'].max()})")
        print(f"  Val:   {len(val):,} records ({val['sales_month'].min()} to {val['sales_month'].max()})")
        
        # Show seasonal coverage
        train_months = sorted(train['sales_month'].dt.month.unique())
        val_months = sorted(val['sales_month'].dt.month.unique())
        print(f"  Train months: {train_months}")
        print(f"  Val months: {val_months}")
        print()
    
    print("BENEFITS OF THIS APPROACH:")
    print("✓ Tests model performance across ALL four seasons")
    print("✓ Validates seasonal pattern learning comprehensively") 
    print("✓ Handles Douyin data gap (2021 Apr-Dec) elegantly")
    print("✓ Enables robust model selection and hyperparameter tuning")
    print("✓ Identifies seasonal biases in model performance")
    
    return splits

# Master function
def complete_feature_engineering_and_prep(df):
    """
    Complete the feature engineering process and prepare for modeling
    """
    print("=" * 70)
    print("STEP 4: STORE CATEGORIZATION & MODEL PREPARATION")
    print("=" * 70)
    
    # Add store categorization
    df = add_store_categorization(df)
    
    # Prepare for modeling
    df, modeling_features = prepare_modeling_dataset(df)
    
    # Create rolling time series splits
    rolling_splits = create_rolling_time_series_splits(df)
    
    print("\n" + "=" * 70)
    print("READY FOR ADVANCED MODELING!")
    print("=" * 70)
    print(f"✓ {len(modeling_features)} features engineered")
    print(f"✓ Store categorization applied") 
    print(f"✓ Rolling time series splits created ({len(rolling_splits)} splits)")
    print(f"✓ Data quality validated")
    print("\nNext: Advanced deep learning models to break 20% MAPE barrier!")
    
    return df, modeling_features, rolling_splits

# Usage in your notebook:
print("Ready to complete feature engineering and model preparation...")
print("Run: df_final, features, rolling_splits = complete_feature_engineering_and_prep(df_advanced)")

In [ ]:
# Save the engineered dataset with all features
import pickle
from datetime import datetime

def save_engineered_dataset(df_final, features, rolling_splits):
    """
    Save the complete engineered dataset and rolling splits for future use
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("=== SAVING ENGINEERED DATASET ===")
    
    # Save main dataset with all features
    main_filename = f"sales_forecast_engineered_dataset_{timestamp}.pkl"
    
    # Create a comprehensive save package
    dataset_package = {
        'df_final': df_final,
        'modeling_features': features,
        'rolling_splits': rolling_splits,
        'metadata': {
            'total_records': len(df_final),
            'total_features': len(features),
            'date_range': {
                'start': df_final['sales_month'].min(),
                'end': df_final['sales_month'].max()
            },
            'platforms': df_final['primary_platform'].unique().tolist(),
            'rolling_splits_info': [
                {
                    'split_num': i+1,
                    'description': description,
                    'train_records': len(train),
                    'val_records': len(val),
                    'train_period': {
                        'start': train['sales_month'].min(),
                        'end': train['sales_month'].max()
                    },
                    'val_period': {
                        'start': val['sales_month'].min(),
                        'end': val['sales_month'].max()
                    }
                }
                for i, (train, val, description) in enumerate(rolling_splits)
            ],
            'feature_categories': {
                'temporal_basic': [col for col in features if any(x in col for x in ['month', 'quarter', 'year', 'days_since'])],
                'temporal_cyclical': [col for col in features if any(x in col for x in ['sin', 'cos'])],
                'promotional': [col for col in features if any(x in col for x in ['promotional', 'distance_to_promo'])],
                'seasonal': [col for col in features if 'seasonal' in col and 'interaction' not in col],
                'lag_features': [col for col in features if 'lag_' in col],
                'rolling_features': [col for col in features if 'rolling_' in col],
                'momentum': [col for col in features if any(x in col for x in ['momentum', 'acceleration', 'pct_change', 'volatility'])],
                'store_behavior': [col for col in features if any(x in col for x in ['store_sales_cv', 'store_sales_range', 'brand_diversity', 'product_diversity'])],
                'store_type': [col for col in features if 'store_type_' in col],
                'brand_market': [col for col in features if any(x in col for x in ['brand_market_share', 'brand_promotional_effectiveness'])],
                'platform_dynamics': [col for col in features if any(x in col for x in ['competing', 'multi_platform', 'platform_preference'])],
                'interactions': [col for col in features if 'interaction' in col],
                'spike_detection': [col for col in features if any(x in col for x in ['spike', 'deviation', 'z_score', 'propensity'])]
            },
            'creation_timestamp': timestamp
        }
    }
    
    # Save as pickle for preserving data types
    with open(main_filename, 'wb') as f:
        pickle.dump(dataset_package, f)
    
    print(f"✓ Complete dataset saved as: {main_filename}")
    
    # Also save a CSV version of the main dataset (without splits) for external tools
    csv_filename = f"sales_forecast_engineered_dataset_{timestamp}.csv"
    df_final.to_csv(csv_filename, index=False)
    print(f"✓ CSV version saved as: {csv_filename}")
    
    # Save feature list as text file for reference
    features_filename = f"modeling_features_{timestamp}.txt"
    with open(features_filename, 'w') as f:
        f.write("MODELING FEATURES FOR SALES FORECASTING\n")
        f.write("="*50 + "\n\n")
        
        for category, feature_list in dataset_package['metadata']['feature_categories'].items():
            if feature_list:
                f.write(f"{category.upper()} ({len(feature_list)} features):\n")
                for feature in feature_list:
                    f.write(f"  - {feature}\n")
                f.write("\n")
        
        f.write(f"TOTAL FEATURES: {len(features)}\n")
        f.write(f"DATASET CREATED: {timestamp}\n")
    
    print(f"✓ Feature documentation saved as: {features_filename}")
    
    # Print summary
    print(f"\nDATASET SAVE SUMMARY:")
    print(f"Total Records: {len(df_final):,}")
    print(f"Total Features: {len(features)}")
    print(f"Rolling Splits: {len(rolling_splits)}")
    for i, (train, val, description) in enumerate(rolling_splits):
        print(f"  Split {i+1}: {len(train):,} train, {len(val):,} val ({description})")
    print(f"Date Range: {df_final['sales_month'].min()} to {df_final['sales_month'].max()}")
    
    return main_filename, csv_filename, features_filename

def load_engineered_dataset(pickle_filename):
    """
    Load the saved engineered dataset
    """
    print(f"Loading engineered dataset from: {pickle_filename}")
    
    with open(pickle_filename, 'rb') as f:
        dataset_package = pickle.load(f)
    
    print("✓ Dataset loaded successfully")
    print(f"Records: {len(dataset_package['df_final']):,}")
    print(f"Features: {len(dataset_package['modeling_features'])}")
    print(f"Rolling Splits: {len(dataset_package['rolling_splits'])}")
    print(f"Created: {dataset_package['metadata']['creation_timestamp']}")
    
    return (
        dataset_package['df_final'],
        dataset_package['modeling_features'],
        dataset_package['rolling_splits'],
        dataset_package['metadata']
    )

# Quick feature summary function
def print_feature_summary(features):
    """
    Print a quick summary of feature categories
    """
    feature_categories = {
        'Temporal': [col for col in features if any(x in col for x in ['month', 'quarter', 'year', 'sin', 'cos', 'days_since'])],
        'Promotional': [col for col in features if any(x in col for x in ['promotional', 'distance_to_promo'])],
        'Seasonal': [col for col in features if 'seasonal' in col],
        'Lag': [col for col in features if 'lag_' in col],
        'Rolling': [col for col in features if 'rolling_' in col],
        'Momentum': [col for col in features if any(x in col for x in ['momentum', 'acceleration', 'pct_change', 'volatility'])],
        'Store': [col for col in features if any(x in col for x in ['store_', 'brand_diversity', 'product_diversity'])],
        'Platform': [col for col in features if any(x in col for x in ['competing', 'multi_platform', 'platform_preference'])],
        'Interactions': [col for col in features if 'interaction' in col],
        'Spikes': [col for col in features if any(x in col for x in ['spike', 'deviation', 'z_score', 'propensity'])]
    }
    
    print("FEATURE SUMMARY:")
    total = 0
    for category, feature_list in feature_categories.items():
        print(f"{category:15s}: {len(feature_list):2d} features")
        total += len(feature_list)
    print(f"{'TOTAL':15s}: {total:2d} features")

# Usage example
print("Ready to save the engineered dataset!")
print("Run: pickle_file, csv_file, features_file = save_engineered_dataset(df_final, features, rolling_splits)")
print("\nTo load later: df, features, rolling_splits, metadata = load_engineered_dataset('filename.pkl')")